# Collation with decision tree

## Find ngrams and positions in witnesses

In [1]:
# two witnesses, with repetition and transposition
w1 = "the red and the black cat"
w2 = "the black and the red cat"

In [2]:
def tokenize_witnesses(w1_string, w2_string):
    '''Return list of witnesses, each represented by a list of tokens'''
    w1_tokens = w1.split()
    w2_tokens = w2.split()
    witnesses = [w1_tokens, w2_tokens]
    return witnesses

In [3]:
witnesses = tokenize_witnesses(w1, w2)
print(witnesses)

[['the', 'red', 'and', 'the', 'black', 'cat'], ['the', 'black', 'and', 'the', 'red', 'cat']]


In [4]:
from collections import defaultdict 

# create a function that creates n-grams and returns the offsets
def compute_ngrams(witness, n):
   output = defaultdict(list)
   for i in range(len(witness)-n+1):
       g = ' '.join(witness[i:i+n])
       output[g].append(i)
   return output

In [5]:
for witness in witnesses:
    print(compute_ngrams(witness, 1))
    print(compute_ngrams(witness, 2))


defaultdict(<class 'list'>, {'the': [0, 3], 'red': [1], 'and': [2], 'black': [4], 'cat': [5]})
defaultdict(<class 'list'>, {'the red': [0], 'red and': [1], 'and the': [2], 'the black': [3], 'black cat': [4]})
defaultdict(<class 'list'>, {'the': [0, 3], 'black': [1], 'and': [2], 'red': [4], 'cat': [5]})
defaultdict(<class 'list'>, {'the black': [0], 'black and': [1], 'and the': [2], 'the red': [3], 'red cat': [4]})


In [6]:
# find bigrams common to the two witnesses
w1_bigrams = compute_ngrams(witnesses[0], 2)
w2_bigrams = compute_ngrams(witnesses[1], 2)
shared_keys = w1_bigrams.keys() & w2_bigrams.keys()
print(shared_keys)

{'the red', 'and the', 'the black'}


In [7]:
from collections import defaultdict

def compute_ngrams_all(witness):
   '''Create a function that creates n-grams and returns the offsets'''
   output = defaultdict(list)
   output2 = {}
   for n in range(1, len(witness) + 1):
       for i in range(len(witness)-n+1):
           g = ' '.join(witness[i:i+n])
           output[g].append(i)
           output2[g] = n
   return output, output2

In [8]:
# compute ngrams for all (both) witnesses
ngram_offset_by_witness_dict = {}
ngram_length = {}
for index, witness in enumerate(witnesses):
    map1, map2 = compute_ngrams_all(witness)
    ngram_offset_by_witness_dict['w' + str(index + 1)] = map1
    ngram_length.update(map2)

for key in ngram_offset_by_witness_dict.keys():
    print(key, ngram_offset_by_witness_dict[key])

print(ngram_length)

w1 defaultdict(<class 'list'>, {'the': [0, 3], 'red': [1], 'and': [2], 'black': [4], 'cat': [5], 'the red': [0], 'red and': [1], 'and the': [2], 'the black': [3], 'black cat': [4], 'the red and': [0], 'red and the': [1], 'and the black': [2], 'the black cat': [3], 'the red and the': [0], 'red and the black': [1], 'and the black cat': [2], 'the red and the black': [0], 'red and the black cat': [1], 'the red and the black cat': [0]})
w2 defaultdict(<class 'list'>, {'the': [0, 3], 'black': [1], 'and': [2], 'red': [4], 'cat': [5], 'the black': [0], 'black and': [1], 'and the': [2], 'the red': [3], 'red cat': [4], 'the black and': [0], 'black and the': [1], 'and the red': [2], 'the red cat': [3], 'the black and the': [0], 'black and the red': [1], 'and the red cat': [2], 'the black and the red': [0], 'black and the red cat': [1], 'the black and the red cat': [0]})
{'the': 1, 'red': 1, 'and': 1, 'black': 1, 'cat': 1, 'the red': 2, 'red and': 2, 'and the': 2, 'the black': 2, 'black cat': 2, '

In [9]:
# find keys shared by *all* witnesses
shared_ngrams = set(ngram_offset_by_witness_dict["w1"].keys())
for value in ngram_offset_by_witness_dict.values():
    shared_ngrams = shared_ngrams.intersection(value.keys())

print(shared_ngrams)

{'red', 'cat', 'and the', 'black', 'the red', 'the', 'the black', 'and'}


In [10]:
# use shared keys to find potential alignments
# NB: works for only two witnesses
# output format: {ngram : [(0,1), (2,3)]}, where 
#   the two entries in each tuple are for witnesses A and B
from collections import defaultdict

potential_alignments = defaultdict(list)
for ngram in shared_ngrams:
    for w1_offset in ngram_offset_by_witness_dict['w1'][ngram]:
        for w2_offset in ngram_offset_by_witness_dict['w2'][ngram]:
            potential_alignments[ngram].append((w1_offset, w2_offset))

print(potential_alignments)

defaultdict(<class 'list'>, {'red': [(1, 4)], 'cat': [(5, 5)], 'and the': [(2, 2)], 'black': [(4, 1)], 'the red': [(0, 3)], 'the': [(0, 0), (0, 3), (3, 0), (3, 3)], 'the black': [(3, 0)], 'and': [(2, 2)]})


## Build decision tree

In [11]:
# Find actual alignments (ngrams with positions in witnesses)
#
# Build dictionary of offset_tuple : list_of_ngrams
#
# Source: https://stackoverflow.com/questions/6190331/how-to-implement-an-ordered-default-dict
#
# agglomerate = collections.OrderedDict()
# for i, x in some_generator():
#     agglomerate.setdefault(i, []).append(x)
#
# Tuples of offsets are keys, sorted; values are lists of ngrams at those offsets

alignments = defaultdict(list)
for key,value in potential_alignments.items():
    for t in value:
        alignments[t].append(key)

print(alignments)

defaultdict(<class 'list'>, {(1, 4): ['red'], (5, 5): ['cat'], (2, 2): ['and the', 'and'], (4, 1): ['black'], (0, 3): ['the red', 'the'], (0, 0): ['the'], (3, 0): ['the', 'the black'], (3, 3): ['the']})


In [12]:
# Create list of tuples (A-position:int, B-position:int, (ngrams), sorted by A position
# Sort ngrams by token count from high to low
# ngrams are in a tuple, rather than list, to make them hashable, and therefore set-able
#
# sorted(alignments.items()) # sorts, but outputs tuples
import collections
sorted_alignments_witness_A = []
for key in sorted(alignments):
    sorted_alignments_witness_A.append((key[0],key[1], tuple(sorted(alignments[key], key=lambda x: ngram_length[x], reverse=True ))))

print(sorted_alignments_witness_A)

[(0, 0, ('the',)), (0, 3, ('the red', 'the')), (1, 4, ('red',)), (2, 2, ('and the', 'and')), (3, 0, ('the black', 'the')), (3, 3, ('the',)), (4, 1, ('black',)), (5, 5, ('cat',))]


In [13]:
# Does it work?
for entry in sorted_alignments_witness_A:
    print(entry)

(0, 0, ('the',))
(0, 3, ('the red', 'the'))
(1, 4, ('red',))
(2, 2, ('and the', 'and'))
(3, 0, ('the black', 'the'))
(3, 3, ('the',))
(4, 1, ('black',))
(5, 5, ('cat',))


In [14]:
# TODO: No need to flip and then flip again
# Sort dictionary of offset_tuple : list_of_ngrams by tuple; Prefer witness B offsets over witness A offsets
# Sort ngrams by length 
#
flipped_witness_offsets = { (k[1], k[0]): v for k,v in alignments.items() }
# print(flipped_witness_offsets)

sorted_alignments_witness_B = []
for key in sorted(flipped_witness_offsets): # flip them back to write A before B
    sorted_alignments_witness_B.append((key[1], key[0], tuple(sorted(flipped_witness_offsets[key], key=lambda x: ngram_length[x], reverse=True ))))

print(sorted_alignments_witness_B)

[(0, 0, ('the',)), (3, 0, ('the black', 'the')), (4, 1, ('black',)), (2, 2, ('and the', 'and')), (0, 3, ('the red', 'the')), (3, 3, ('the',)), (1, 4, ('red',)), (5, 5, ('cat',))]

### A decision tree node stores the following properties:
* unique id
* current location in witness A
* current location in witness B
* parent (except root)
* children
* aligned patterns
* potential alignments by A
* potential alignments by B

Edges are not stored separately; edge information is represented by parent and children properties of node

### Implement with nested dictionaries:
* {id:int : properties:dict} for each node
* properties (7) : 
  * current-location-in-A:int (id of node)
  * current-location-in-B:int (id of node)
  * parent : id:int
  * children : [id:int]
  * aligned-patterns : [(offsetA:int, offsetB:int, ngram:str)]
  * potential-alignments-by-A : [(offsetA:int, offsetB:int, ngram:str)]
  * potential-alignments-by-B : [(offsetB:int, offsetA:int, ngram:str)]

In [15]:
# Create outer dictionary for decision tree nodes
decision_tree = {}

# Add root node
decision_tree[0] = {}

# Supply properties for root node
decision_tree[0]['current-location-in-A'] = -1 # 0 would be the first position
decision_tree[0]['current-location-in-B'] = -1
decision_tree[0]['parent'] = None # integer
decision_tree[0]['children'] = []
decision_tree[0]['aligned-patterns'] = []
decision_tree[0]['potential-alignments-by-A'] = sorted_alignments_witness_A
decision_tree[0]['potential-alignments-by-B'] = sorted_alignments_witness_B

In [16]:
# Take a look
decision_tree[0]

{'current-location-in-A': -1,
 'current-location-in-B': -1,
 'parent': None,
 'children': [],
 'aligned-patterns': [],
 'potential-alignments-by-A': [(0, 0, ('the',)),
  (0, 3, ('the red', 'the')),
  (1, 4, ('red',)),
  (2, 2, ('and the', 'and')),
  (3, 0, ('the black', 'the')),
  (3, 3, ('the',)),
  (4, 1, ('black',)),
  (5, 5, ('cat',))],
 'potential-alignments-by-B': [(0, 0, ('the',)),
  (3, 0, ('the black', 'the')),
  (4, 1, ('black',)),
  (2, 2, ('and the', 'and')),
  (0, 3, ('the red', 'the')),
  (3, 3, ('the',)),
  (1, 4, ('red',)),
  (5, 5, ('cat',))]}

In [17]:
# Traverse from left to right, so children of root are all
#   potential alignments at 0 in one or both witnesses
#   In this case: four options, three of which are unique
#   First item of first tuple is nearest position, so find all tuples with that value

nearest_A_matches = [item for item in decision_tree[0]['potential-alignments-by-A'] if item[0] == decision_tree[0]['potential-alignments-by-A'][0][0]]
nearest_B_matches = [item for item in decision_tree[0]['potential-alignments-by-B'] if item[1] == decision_tree[0]['potential-alignments-by-B'][0][1]]
nearest_matches = {item for item in nearest_A_matches + nearest_B_matches}
print(nearest_matches)

{(0, 0, ('the',)), (3, 0, ('the black', 'the')), (0, 3, ('the red', 'the'))}


In [18]:
# Update root node to add children
# Within children:
#   Add parent
#   Update current location in A and B
#   Update aligned patterns
#   Update potential alignments for A and B
#
# For each of the (three) items in the set of nearest_matches, create a child
# Assign consecutive id values by counting size of dictionary = could break with multithreading

for child in nearest_matches:
    child = (child[0], child[1], child[2][0]) # prune ngrams to keep only longest
    current_ngram_length = ngram_length[child[2]]
    id = len(decision_tree)
    decision_tree[id] = {}
    decision_tree[id]['current-location-in-A'] = child[0] + current_ngram_length - 1
    decision_tree[id]['current-location-in-B'] = child[1] + current_ngram_length - 1
    decision_tree[id]['parent'] = 0
    decision_tree[id]['children'] = []
    aligned_patterns = decision_tree[0]['aligned-patterns'].copy()
    aligned_patterns.append(child)
    decision_tree[id]['aligned-patterns'] = aligned_patterns
    
    # TODO: Because lists are sorted, once one is far enough to the right to avoid transposition,
    #   all following ones are also okay (in this case, though, must process A and B separately)
    # Similarly, once one ngram is safe, all shorter ones are also safe, and don't need to be checked
    decision_tree[id]['potential-alignments-by-A'] = []
    for p in decision_tree[0]['potential-alignments-by-A']: # throw away potentials where no ngram can work
        if p[0] > decision_tree[id]['current-location-in-A'] and p[1] > decision_tree[id]['current-location-in-B']:
            new_ngram_list = [] # check each ngram in each remaining potential starting from the longest (left)
            for q in p[2]:
                q_ngram_length = ngram_length[q] # length of current ngram inside current potential
                if p[0] + q_ngram_length > decision_tree[id]['current-location-in-A'] and p[1] + q_ngram_length > decision_tree[id]['current-location-in-B']: # keep the ones that don't entail transposition
                    new_ngram_list.append(q)
            decision_tree[id]['potential-alignments-by-A'].append((p[0], p[1], tuple(new_ngram_list))) # add potential, with pruned list of ngrams, to new potentials

    # potentials by A and B are the same tuples, but sorted differently
    decision_tree[id]['potential-alignments-by-B'] = sorted(decision_tree[id]['potential-alignments-by-A'], key=lambda x: (x[1], x[0]))

    decision_tree[0]['children'].append(id)

# take a look
decision_tree

{0: {'current-location-in-A': -1,
  'current-location-in-B': -1,
  'parent': None,
  'children': [1, 2, 3],
  'aligned-patterns': [],
  'potential-alignments-by-A': [(0, 0, ('the',)),
   (0, 3, ('the red', 'the')),
   (1, 4, ('red',)),
   (2, 2, ('and the', 'and')),
   (3, 0, ('the black', 'the')),
   (3, 3, ('the',)),
   (4, 1, ('black',)),
   (5, 5, ('cat',))],
  'potential-alignments-by-B': [(0, 0, ('the',)),
   (3, 0, ('the black', 'the')),
   (4, 1, ('black',)),
   (2, 2, ('and the', 'and')),
   (0, 3, ('the red', 'the')),
   (3, 3, ('the',)),
   (1, 4, ('red',)),
   (5, 5, ('cat',))]},
 1: {'current-location-in-A': 0,
  'current-location-in-B': 0,
  'parent': 0,
  'children': [],
  'aligned-patterns': [(0, 0, 'the')],
  'potential-alignments-by-A': [(1, 4, ('red',)),
   (2, 2, ('and the', 'and')),
   (3, 3, ('the',)),
   (4, 1, ('black',)),
   (5, 5, ('cat',))],
  'potential-alignments-by-B': [(4, 1, ('black',)),
   (2, 2, ('and the', 'and')),
   (3, 3, ('the',)),
   (1, 4, ('red